In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mvf_bto.data_loading import load_data
from mvf_bto.constants import * 
from mvf_bto.models.baseline_lstm import BaselineLSTM
from mvf_bto.preprocessing.discharge import create_discharge_inputs
from mvf_bto.preprocessing.charge import create_charge_inputs

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError

import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go

## Loading Data

In [3]:
# data_path = "/Users/anoushkabhutani/PycharmProjects/10701-mvf-bto/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"
data_path = "/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto-backup/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"

In [4]:
data = load_data(file_path=data_path, num_cells=9)

100%|██████████| 9/9 [01:26<00:00,  9.65s/it]


## Preprocessing to create model inputs and targets

In [5]:
train_split = 0.7
test_split = 0.2
# by default uses validation_split = 1 - (train_split + test_split)

In [6]:
cell_id='b1c8'
policy=data[cell_id]["charge_policy"]
print(policy)

5.4C(40%)-3.6C


In [7]:
print(data.keys())

dict_keys(['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4', 'b1c5', 'b1c6', 'b1c7', 'b1c8'])


In [32]:
datasets = create_charge_inputs(data, train_split, test_split)

100%|██████████| 878/878 [00:09<00:00, 91.16it/s] 


In [33]:
train_cell_ids = list(datasets['original_train']['Cell'].unique())
test_cell_ids = list(datasets['original_test']['Cell'].unique())
val_cell_ids = list(datasets['original_val']['Cell'].unique())
print(train_cell_ids,test_cell_ids,val_cell_ids)

['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4', 'b1c5'] ['b1c6'] ['b1c7', 'b1c8']


In [34]:
for cell in train_cell_ids:
    temp = datasets['original_train'][datasets['original_train'].Cell==cell]
    print(len(temp.Cycle.unique()))

1126
1083
1090
420
1172
1069


In [35]:
# fig=go.Figure()
# cell='b1c3'
# for cycle in range(1,1200,50):
#     temp=datasets['original_train'][(datasets['original_train'].Cell==cell) & (datasets['original_train'].Cycle==cycle)]
#     fig.add_trace(go.Scatter(x=temp['Qc'],
#                             y=temp['temp'],
#                             mode="markers"))
# fig.show()

In [36]:
# fig=go.Figure()
# cell='b1c1'
# for cycle in range(1,1200,50):
#     temp=datasets['original_train'][(datasets['original_train'].Cell==cell) & (datasets['original_train'].Cycle==cycle)]
#     fig.add_trace(go.Scatter(x=temp['Qc'],
#                             y=temp['temp'],
#                             mode="markers"))
# fig.show()

## Train Model

In [37]:
window_length = datasets["X_train"].shape[1]
n_features = datasets["X_train"].shape[2]
batch_size=datasets["batch_size"]
batch_input_shape = (datasets["batch_size"], window_length, n_features)
n_outputs = datasets["y_train"].shape[-1]
print(window_length, n_features, batch_input_shape, batch_size, n_outputs,datasets["X_train"].shape[0]//batch_size)
print(datasets["y_train"].shape)
print(datasets["X_train"].shape)

3 7 (45, 3, 7) 45 2 5960
(268200, 1, 2)
(268200, 3, 7)


In [38]:
original_df=datasets['train_dfs']
original_df=original_df[original_df.Cell=='b1c0']
original_df=original_df[original_df.Cycle==5/MAX_CYCLE]
original_df.describe()

,Q_eval,V_norm,T_norm,Cycle,C_rate1,SOC1,C_rate2
count,50.000000,50.000000,50.000000,50.000000,50.0,5.000000e+01,50.0
mean,0.644499,0.934217,0.541714,0.002174,3.6,8.000000e-01,3.6
std,0.357115,0.148758,0.070011,0.000000,0.0,2.242989e-16,0.0
min,0.000000,0.077631,0.410924,0.002174,3.6,8.000000e-01,3.6
25%,0.326530,0.933418,0.500619,0.002174,3.6,8.000000e-01,3.6
50%,0.817086,0.943314,0.540967,0.002174,3.6,8.000000e-01,3.6
75%,0.890180,0.990859,0.599167,0.002174,3.6,8.000000e-01,3.6
max,0.984854,1.062697,0.671573,0.002174,3.6,8.000000e-01,3.6


In [39]:
skip=500
fig = go.Figure()
train_cell_id_idx = 0
last_cycle = 1
current_cycle = 1
pallete = plotly.colors.qualitative.Dark24*(len(datasets["X_train"])//skip)

for i in range(0, len(datasets["X_train"]), batch_size * skip):

    last_cycle = current_cycle
    current_cycle = int(datasets["X_train"][i : i + batch_size][0][0][3]*MAX_CYCLE)
    if last_cycle> current_cycle:
        train_cell_id_idx += 1
    cell_id = train_cell_ids[train_cell_id_idx]

    original_df = datasets["original_train"]
    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    df = datasets['train_dfs']
    df = df[df.Cycle == current_cycle/MAX_CYCLE]
    df = df[df.Cell==cell_id]
    REFERENCE_CHARGE_CAPACITIES=df['Q_eval']
    # original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    # original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]
    original_df = original_df[original_df.I > MIN_CHARGE_CURRENT]

    original_df['Qc'] = original_df['Qc'] / MAX_CHARGE_CAPACITY

    df_true = pd.DataFrame(datasets["y_train"][i : i + batch_size, 0])
    
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            # x=np.linspace(0,1,201)[-batch_size:],
            y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="lines+markers",
            # name = f"Curve {i//batch_size+1}",
            name = f"{cell_id} Cycle {current_cycle}",
            line_color=pallete[i//skip]
        )
    )

    if len(original_df)< 2:
        continue
    q_new = original_df['Qc'].values
    t_new = original_df['t'].values
    nf_steps=1
    odf=original_df
    fig.add_trace(
        go.Scatter(
            x=odf['Qc'],
            y=odf['V'],
            showlegend=True,
            mode="markers+lines",
            name = f"{cell_id} Cycle {current_cycle} real",
            # line_color=pallete[i]
        )
    )

    # cell_id = train_cell_ids[train_cell_id_idx]
    # original_df = datasets["original_train"]
    # original_df= original_df[original_df.Cycle==current_cycle]
    # original_df= original_df[original_df.Cell==cell_id]
    # original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    # original_df = original_df[original_df.I > MIN_CHARGE_CURRENT]

    # original_df['Qd'] = (original_df['Qd']-original_df['Qd'].min())/(original_df['Qd'].max()-original_df['Qd'].min())

fig.update_yaxes(title="Voltage [V]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")

In [40]:
from scipy.interpolate import interp1d

In [44]:
skip = 100
test_cell_id_idx = 0
last_cycle = 1
current_cycle = 1
# opacity_list = [1, 0.6, 0.3]
fig = go.Figure()

for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, 0, 0])
    
    last_cycle = current_cycle
    current_cycle = int(datasets["X_test"][i : i + batch_size][0][0][3]*MAX_CYCLE)
    # print(current_cycle)
    # if current_cycle>740:
    #     continue
    if last_cycle> current_cycle:
        test_cell_id_idx += 1
    cell_id = test_cell_ids[test_cell_id_idx]
    original_df = datasets["original_test"]
    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    df = datasets['test_dfs']
    df = df[df.Cycle == current_cycle/MAX_CYCLE]
    df = df[df.Cell==cell_id]
    REFERENCE_CHARGE_CAPACITIES=df['Q_eval']    
    # original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    # original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]
    original_df = original_df[original_df.I > MIN_CHARGE_CURRENT]

    original_df['Qc'] = (original_df['Qc'])/(MAX_CHARGE_CAPACITY)

    if len(original_df)< 2:
        continue
    q_new = original_df['Qc'].values
    t_new = original_df['t'].values
    time_interpolator = interp1d(x = q_new,  y = t_new, 
    # fill_value="extrapolate"
    )
    
    ref_capacities_wrt_nf = REFERENCE_CHARGE_CAPACITIES[window_length:-2]
    # print(window_length)
    # print(len(ref_capacities_wrt_nf), len(df_true[0]))
    prediction_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                        y = df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                                        # fill_value="extrapolate"
                                         )
        
    q_j = q_new[q_new>= min(REFERENCE_CHARGE_CAPACITIES[window_length:-2])]
    q_j = q_j[q_j<= max(REFERENCE_CHARGE_CAPACITIES[window_length:-2])]
    t_interp = time_interpolator(REFERENCE_CHARGE_CAPACITIES[window_length:-2])
    q_j0 = q_j
    V_pred = prediction_interpolator(q_j)
    fig.add_trace(
        go.Scatter(
                x=time_interpolator(ref_capacities_wrt_nf),
                # x=q_j,
                # y=time_interpolator(q_j),
                # y=V_pred,
                y = df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                showlegend=True,
                mode="lines+markers",
                # line_dash="dash",
                name = "intp",
                # marker_color=pallete[i],
                # opacity=opacity_list[j]
            )
        )
        
    # fig.add_trace(
    #         go.Scatter(
    #             x=time_interpolator(ref_capacities_wrt_nf),
    #             y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
    #             showlegend=True,
    #             mode="markers",
    #             name = f"Predictions Forecast Horizon {j+1}",
    #             marker_color=pallete[i],
    #             marker_symbol = symbol_list[j],
    #             marker_size=8,
    #             marker_opacity=opacity_list[j]
    #         )
    #     )

    if len(original_df)< 2:
        continue
    q_new = original_df['Qc'].values
    t_new = original_df['t'].values
    nf_steps=1
    odf=original_df
    fig.add_trace(
        go.Scatter(
            x=odf['t'],
            y=odf['V'],
            showlegend=True,
            mode="lines+markers",
            # name = f"{cell_id} Cycle {current_cycle}",
            name = f"{cell_id}cycle{current_cycle}"
            # line_color=pallete[i]
        )
    )


fig.update_yaxes(title="Voltage [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

In [49]:
skip = 100
val_cell_id_idx = 0
last_cycle = 1
current_cycle = 1
# opacity_list = [1, 0.6, 0.3]
fig = go.Figure()

for i in range(0, len(datasets["X_val"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_val"][i : i + batch_size][:, 0, 0])
    
    last_cycle = current_cycle
    current_cycle = int(datasets["X_val"][i : i + batch_size][0][0][3]*MAX_CYCLE)
    # print(current_cycle)
    # if current_cycle>740:
    #     continue
    if last_cycle> current_cycle:
        val_cell_id_idx += 1
    cell_id = val_cell_ids[val_cell_id_idx]
    original_df = datasets["original_val"]
    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    df = datasets['val_dfs']
    df = df[df.Cycle == current_cycle/MAX_CYCLE]
    df = df[df.Cell==cell_id]
    REFERENCE_CHARGE_CAPACITIES=df['Q_eval']    
    # original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    # original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]
    original_df = original_df[original_df.I > MIN_CHARGE_CURRENT]

    original_df['Qc'] = (original_df['Qc'])/(MAX_CHARGE_CAPACITY)

    if len(original_df)< 2:
        continue
    q_new = original_df['Qc'].values
    t_new = original_df['t'].values
    time_interpolator = interp1d(x = q_new,  y = t_new, 
    # fill_value="extrapolate"
    )
    
    ref_capacities_wrt_nf = REFERENCE_CHARGE_CAPACITIES[window_length:-2]
    # print(window_length)
    # print(len(ref_capacities_wrt_nf), len(df_true[0]))
    prediction_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                        y = df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                                        # fill_value="extrapolate"
                                         )
        
    q_j = q_new[q_new>= min(REFERENCE_CHARGE_CAPACITIES[window_length:-2])]
    q_j = q_j[q_j<= max(REFERENCE_CHARGE_CAPACITIES[window_length:-2])]
    t_interp = time_interpolator(REFERENCE_CHARGE_CAPACITIES[window_length:-2])
    q_j0 = q_j
    V_pred = prediction_interpolator(q_j)
    fig.add_trace(
        go.Scatter(
                x=time_interpolator(ref_capacities_wrt_nf),
                # x=q_j,
                # y=time_interpolator(q_j),
                # y=V_pred,
                y = df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                showlegend=True,
                mode="lines+markers",
                # line_dash="dash",
                name = f"intp{cell_id}cycle{current_cycle}",
                # marker_color=pallete[i],
                # opacity=opacity_list[j]
            )
        )
        
    # fig.add_trace(
    #         go.Scatter(
    #             x=time_interpolator(ref_capacities_wrt_nf),
    #             y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
    #             showlegend=True,
    #             mode="markers",
    #             name = f"Predictions Forecast Horizon {j+1}",
    #             marker_color=pallete[i],
    #             marker_symbol = symbol_list[j],
    #             marker_size=8,
    #             marker_opacity=opacity_list[j]
    #         )
    #     )

    if len(original_df)< 2:
        continue
    q_new = original_df['Qc'].values
    t_new = original_df['t'].values
    nf_steps=1
    odf=original_df
    fig.add_trace(
        go.Scatter(
            x=odf['t'],
            y=odf['V'],
            showlegend=True,
            mode="lines+markers",
            # name = f"{cell_id} Cycle {current_cycle}",
            name = f"{cell_id}cycle{current_cycle}"
            # line_color=pallete[i]
        )
    )


fig.update_yaxes(title="Voltage [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

In [ ]:
len(datasets["y_train"][i : i + batch_size][:, 0, 0])

In [ ]:
print(datasets["X_train"][0])

In [ ]:

model = BaselineLSTM(batch_input_shape=batch_input_shape, n_outputs=n_outputs)

In [ ]:
model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])

In [ ]:
skip=100
fig = go.Figure()
pallete = plotly.colors.qualitative.Dark24*(len(datasets["X_train"])//skip)

for i in range(0, len(datasets["X_train"]), batch_size * skip):

    df_true = pd.DataFrame(datasets["y_train"][i : i + batch_size, 0])
    
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="lines+markers",
            name = f"Curve {i//batch_size+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Voltage [V]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")

In [ ]:
model = BaselineLSTM(batch_input_shape=batch_input_shape, n_outputs=n_outputs)
model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])

es = EarlyStopping(
    monitor="val_mean_squared_error",
    min_delta=0,
    patience=10,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
)

history = model.fit(
    datasets["X_train"],
    datasets["y_train"],
    validation_data=(datasets["X_val"], datasets["y_val"]),
    epochs=150,
    batch_size=datasets["batch_size"],
    shuffle=False,
    callbacks=[es],
    verbose=1,
)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.linspace(1, 50),
        y=history.history["loss"],
        showlegend=False,
        mode="markers+lines",
    )
)
fig.update_xaxes(title="Epochs")
fig.update_yaxes(title="Loss (MSE)")

## Parity Plot of Training Error

In [ ]:
pd.DataFrame(datasets["y_train"][:,0,:])

In [ ]:
# random plotting traing error at some interval = skip to not make the plot rendering too slow
batch_size = datasets["batch_size"]
skip = 600

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1.5], y=[0, 1.5], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_train"][i : i + batch_size], verbose=0, batch_size=batch_size)[:,0,:])
    df_train = pd.DataFrame(datasets["y_train"][:,0,:])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1.5], y=[0, 1.5], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_train"][i : i + batch_size], verbose=0, batch_size=batch_size)[:,0,:])
    df_train = pd.DataFrame(datasets["y_train"][:,0,:])
    fig.add_trace(
        go.Scatter(
            x=df_pred[1].values,
            y=df_train[1].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Temperature Target")
fig.update_xaxes(title="Normalized Temperature Prediction")

## Parity Plot of Test Error

In [ ]:
skip = 50

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1.5], y=[0, 1.5], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0, batch_size=batch_size)[:,0,:])
    df_train = pd.DataFrame(datasets["y_test"][:,0,:])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1.5], y=[0, 1.5], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0, batch_size=batch_size)[:,0,:])
    df_train = pd.DataFrame(datasets["y_test"][:,0,:])
    fig.add_trace(
        go.Scatter(
            x=df_pred[1].values,
            y=df_train[1].values,
            showlegend=False,
            mode="markers+lines",
        ))
fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Temperature Prediction")

## True vs Predicted Traces (Test Set)

In [ ]:
skip = 20

pallete = plotly.colors.qualitative.Dark24*(len(datasets["X_test"])//skip)

fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0, batch_size=batch_size)[:,0,:])
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size][:,0,:])
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="markers",
            name = f"Predicted Curve {i+1}",
            marker_color=pallete[i//skip]
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="lines",
            name = f"True Curve {i+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Voltage [V]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")

In [ ]:
fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], batch_size=batch_size, verbose=0)[:,0,:])
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size][:,0,:])
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_pred[1].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
            showlegend=True,
            mode="markers",
            name = f"Predicted Curve {i+1}",
            marker_color=pallete[i//skip]
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CHARGE_CAPACITIES[-batch_size:],
            y=df_true[1].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
            showlegend=True,
            mode="lines",
            name = f"True Curve {i+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Temperature [°C]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")

In [ ]:
symbol_list = ["circle-open", "circle", "triangle-up"]
pallete = plotly.colors.qualitative.Dark24 + plotly.colors.qualitative.T10
pallete = pallete*70000

In [ ]:
skip = 100
test_cell_id_idx = 0
last_cycle = 1
current_cycle = 1
opacity_list = [1, 0.6, 0.3]
fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, 0, 0])
    
    last_cycle = current_cycle
    current_cycle = int(datasets["X_test"][i : i + batch_size][0][0][3]*MAX_CYCLE)

    if last_cycle> current_cycle:
        test_cell_id_idx += 1
        
    cell_id = test_cell_ids[test_cell_id_idx]
    original_df = datasets["original_test"]

    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    # original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    # original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

    original_df['Qc'] = (original_df['Qc']-original_df['Qc'].min())/(original_df['Qc'].max()-original_df['Qc'].min())

    if len(original_df)< 2:
        continue
    q_new = original_df["Qc"].values
    t_new = original_df['t'].values
    time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    for j in range(nf_steps):
        df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0, batch_size=batch_size)[:, j, 0])
    
        ref_capacities_wrt_nf = REFERENCE_CHARGE_CAPACITIES[window_length+j:-nf_steps+j-1]

        prediction_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                                          )
        
        q_j = q_new[q_new>= min(REFERENCE_CHARGE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        q_j = q_j[q_j<= max(REFERENCE_CHARGE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        t_interp = time_interpolator(REFERENCE_CHARGE_CAPACITIES[window_length:-nf_steps-1])
        if j==0:
            q_j0 = q_j
        V_pred = prediction_interpolator(q_j)
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(q_j),
                y=V_pred,
                showlegend=True,
                mode="lines",
                line_dash="dash",
                name = "Interpolated Predictions",
                marker_color=pallete[i//skip],
                opacity=opacity_list[j]
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(ref_capacities_wrt_nf),
                y=df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predictions Forecast Horizon {j+1}",
                marker_color=pallete[i//skip],
                marker_symbol = symbol_list[j],
                marker_size=8,
                marker_opacity=opacity_list[j]
            )
        )

    odf = original_df
    # odf = original_df[original_df.Qd > q_j0.min()]
    # odf = odf[odf.Qd < q_j.max()]
    fig.add_trace(
        go.Scatter(
            x=odf['t'],
            y=odf['V'],
            showlegend=True,
            mode="lines+markers",
            name = f"{cell_id} Cycle {current_cycle}",
            line_color=pallete[i//skip]
        )
    )


fig.update_yaxes(title="Voltage [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")

In [ ]:
from tensorflow import keras
model.save("/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto/pre_train/charge/v2")

In [ ]:
skip = 100
test_cell_id_idx = 0
last_cycle = 1
current_cycle = 1
opacity_list = [1, 0.6, 0.3]
fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, 0, 1])
    
    last_cycle = current_cycle
    current_cycle = int(datasets["X_test"][i : i + batch_size][0][0][3]*MAX_CYCLE)

    if last_cycle> current_cycle:
        test_cell_id_idx += 1
        
    cell_id = test_cell_ids[test_cell_id_idx]
    original_df = datasets["original_test"]

    original_df= original_df[original_df.Cycle==current_cycle]
    original_df= original_df[original_df.Cell==cell_id]
    # original_df = original_df[original_df.I < MAX_DISCHARGE_CURRENT]
    # original_df = original_df[original_df.I > MIN_DISCHARGE_CURRENT]

    original_df['Qc'] = (original_df['Qc']-original_df['Qc'].min())/(original_df['Qc'].max()-original_df['Qc'].min())

    if len(original_df)< 2:
        continue
    q_new = original_df["Qc"].values
    t_new = original_df['t'].values
    time_interpolator = interp1d(x = q_new,  y = t_new, fill_value="extrapolate")
    
    for j in range(nf_steps):
        df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0, batch_size=batch_size)[:, j, 1])
    
        ref_capacities_wrt_nf = REFERENCE_CHARGE_CAPACITIES[window_length+j:-nf_steps+j-1]

        prediction_interpolator = interp1d(x = ref_capacities_wrt_nf, 
                                           y = df_pred[0].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
                                          )
        
        q_j = q_new[q_new>= min(REFERENCE_CHARGE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        q_j = q_j[q_j<= max(REFERENCE_CHARGE_CAPACITIES[window_length+j:-nf_steps+j-1])]
        t_interp = time_interpolator(REFERENCE_CHARGE_CAPACITIES[window_length:-nf_steps-1])
        if j==0:
            q_j0 = q_j
        V_pred = prediction_interpolator(q_j)
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(q_j),
                y=V_pred,
                showlegend=True,
                mode="lines",
                line_dash="dash",
                name = "Interpolated Predictions",
                marker_color=pallete[i//skip],
                opacity=opacity_list[j]
            )
        )
        
        fig.add_trace(
            go.Scatter(
                x=time_interpolator(ref_capacities_wrt_nf),
                y=df_pred[0].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predictions Forecast Horizon {j+1}",
                marker_color=pallete[i//skip],
                marker_symbol = symbol_list[j],
                marker_size=8,
                marker_opacity=opacity_list[j]
            )
        )

    odf = original_df
    # odf = original_df[original_df.Qd > q_j0.min()]
    # odf = odf[odf.Qd < q_j.max()]
    fig.add_trace(
        go.Scatter(
            x=odf['t'],
            y=odf['temp'],
            showlegend=True,
            mode="lines+markers",
            name = f"{cell_id} Cycle {current_cycle}",
            line_color=pallete[i//skip]
        )
    )


fig.update_yaxes(title="Voltage [V]", showgrid=True)
fig.update_xaxes(title="Times (min)", showgrid=True)
fig.update_layout(template="simple_white")